In [1]:
from PIL import Image
import numpy as np
import os

In [3]:
dir_train = '../Data/Train_dataset'
dir_mask = '../Data/Mask'
file_train = os.listdir(dir_train)

In [ ]:
idx=1
sliding_window = 20

In [ ]:
idfile = 1
for i in range(len(file_train)):
    image = file_train[i]
    mask = file_train[i].replace('.jpg', '_classimg_nonconvex.png')
    if os.path.isfile(f'{dir_train}/{image}') == False or os.path.isfile(f'{dir_mask}/{mask}') == False:
      print(f'pass {dir_train}/{image}')
      continue

    train_img = np.array(Image.open(f'{dir_train}/{image}'))
    mask_img = np.array(Image.open(f'{dir_mask}/{mask}'))

    # Calculate the new dimensions for the cropped image
    new_height = np.shape(train_img)[0] // sliding_window
    new_width = np.shape(train_img)[1] // sliding_window

    # Crop the image
    for h in range(0, np.shape(train_img)[0] - new_height+1, new_height):
      for w in range(0, np.shape(train_img)[1] - new_width+1, new_width):
        train_subimg = train_img[h:h+new_height, w:w+new_width]
        values, counts = np.unique(train_subimg, return_counts=True)
        if np.min(values) >= 205:
            continue
        ivalue = 1
        value_check = 205
        while True:
            if value_check not in values:
                if value_check == 254:
                    print(f'find 205: {file_train[i]}, ({h},{w}) {value_check} ,{ivalue}')
                    ivalue = -1
                value_check +=ivalue
            else:
                break
        percentate = np.sum(counts[np.where(values == value_check)[0][0]:])/(new_height* new_width* 3)
        if percentate >= 0.85:
            continue

        if idx == 3000:
          idfile+=1
          idx = 1

        if os.path.isdir(f'./Data/Image_X20/Image{idfile}') == False:
          os.mkdir(f'./Data/Image_X20/Image{idfile}')
          os.mkdir(f'./Data/Image_X20/Mask{idfile}')
        Image.fromarray(train_subimg).save(f'./Data/Image_X20/Image{idfile}/{idx}_{image}')
        mask_subimg = mask_img[h:h+new_height, w:w+new_width]
        Image.fromarray(mask_subimg).save(f'./Data/Image_X20/Mask{idfile}/{idx}_{mask}')
        idx +=1